Pipeline = 可组合的步骤 + 上下文 + 控制流
条件 Pipeline 本质是：根据 Context（运行时状态）动态决定下一步执行哪些 Command
对应三类常见需求：

In [ ]:
from dataclasses import dataclass
from playwright.async_api import Page


@dataclass
class CrawlContext:
    page: Page
    data: dict


class PWCommand:
    async def run(self, ctx: CrawlContext) -> CrawlContext: ...


class Condition:
    async def check(self, ctx: CrawlContext) -> bool: ...


class IfCommand(PWCommand):
    def __init__(
        self,
        condition: Condition,
        then_cmds: list[PWCommand],
        else_cmds: list[PWCommand] | None = None,
    ):
        self.condition = condition
        self.then_cmds = then_cmds
        self.else_cmds = else_cmds or []

    async def run(self, ctx: CrawlContext) -> CrawlContext:
        cmds = self.then_cmds if await self.condition.check(ctx) else self.else_cmds
        for cmd in cmds:
            ctx = await cmd.run(ctx)
        return ctx


class IsLoggedIn(Condition):
    async def check(self, ctx: CrawlContext) -> bool:
        return await ctx.page.locator(".user-avatar").is_visible()


pipeline = [
    NavigateCommand("https://www.bilibili.com"),
    IfCommand(
        condition=IsLoggedIn(),
        then_cmds=[],
        else_cmds=[
            ClickCommand("#login"),
            FillCommand("#username", "xxx"),
            FillCommand("#password", "yyy"),
            ClickCommand("#submit"),
        ],
    ),
]
